In [3]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import re

seed = 101

Load the dataset from the previous notebook.

In [2]:
with sql.connect('../data/toxic.db') as conn:
    df = pd.read_sql_query('select * from toxic', conn)
df.head()

,rev_id,comment,year,logged_in,ns,sample,split,min,max,avg,y
0,2232.0,This:\n:One can make an analogy in mathematica...,2002,1,article,random,train,-1.0,1.0,0.4,0
1,4216.0,"""\n\n:Clarification for you (and Zundark's ri...",2002,1,user,random,train,0.0,2.0,0.5,0
2,8953.0,Elected or Electoral? JHK,2002,0,article,random,test,0.0,1.0,0.1,0
3,26547.0,"""This is such a fun entry. Devotchka\n\nI on...",2002,1,article,random,train,0.0,2.0,0.6,0
4,28959.0,Please relate the ozone hole to increases in c...,2002,1,article,random,test,-1.0,1.0,0.2,0


Remember to isolate the train, dev, and test sets.

In [51]:
idx_train = df['split'] == 'train'
idx_dev = df['split'] == 'dev'
idx_test = df['split'] == 'test'

Let's start things off with a pretty basic model to serve as a baseline: a CountVectorizer and a MultinomialNB. We'll start with the tokenizer we developed in the exploration step.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def tokenizer(text):
    return re.findall(r'[a-z0-9]+', text)

vect = CountVectorizer(tokenizer=tokenizer)
clf = MultinomialNB()

In [10]:
X_train = df.loc[idx_train, 'comment'].values
X_dev = df.loc[idx_dev, 'comment'].values

y_train = df.loc[idx_train, 'y'].values
y_dev = df.loc[idx_dev, 'y'].values

X_train_vect = vect.fit_transform(X_train)
X_dev_vect = vect.transform(X_dev)

In [11]:
clf.fit(X_train_vect, y_train)
y_pred = clf.predict(X_dev_vect)
np.mean(y_dev==y_pred)

0.8969434760956175

Nice, we nearly got 90% on the dev set. Is accuracy a good metric for this problem though?

In [12]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_dev, y_pred)

array([[25211,  1039],
       [ 2272,  3606]], dtype=int64)

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_dev, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94     26250
           1       0.78      0.61      0.69      5878

   micro avg       0.90      0.90      0.90     32128
   macro avg       0.85      0.79      0.81     32128
weighted avg       0.89      0.90      0.89     32128



In [23]:
idx_error = (y_dev != y_pred) & (y_dev == 1)
print(X_dev[idx_error][3])



==Werewolf: The Whatever==

:No reason for these minor elements of W:tF backstory to have their own articles.

Was that a deliberate typo? Either way, it's quite funny ).  


Since we have a pretty large imbalance and the positive class is low, let's use F1 as the evaluation metric.

In [24]:
from sklearn.metrics import f1_score

f1_score(y_dev, y_pred)

0.685355887104438

So how do we efficiently tune our hyperparameters? We can use GridSearchCV or RandomizedSearchCV, but we have a pre-defined dev set so we need to use some tricks to override the normal behavior. This is actually pretty standard for large-scale NLP problems. Cross-validation is preferred, but often not feasible for large datasets.

```
For some datasets, a pre-defined split of the data into training- and validation fold or into several cross-validation folds already exists. Using PredefinedSplit it is possible to use these folds e.g. when searching for hyperparameters.

For example, when using a validation set, set the test_fold to 0 for all samples that are part of the validation set, and to -1 for all other samples.
```

In [38]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.pipeline import Pipeline

df_train_dev = df[df['split'].map(lambda x: x in {'train', 'dev'})].copy()
idx = np.where(df_train_dev['split'] == 'dev', 0, -1) # See documentation above

ps = PredefinedSplit(idx)

In [67]:
pipe = Pipeline([('vect', CountVectorizer(tokenizer=tokenizer)),
                 ('clf', MultinomialNB())])
param_grid = {'vect__ngram_range':[(1,1),(1,2)], 'vect__min_df':[1, 2, 5, 10, 20],
              'vect__stop_words':['english', None], 'clf__alpha':[0.1, 0.2, 0.5, 1.0]}
gs = GridSearchCV(pipe, param_grid, scoring='f1', n_jobs=4, cv=ps, verbose=2)
gs.fit(df_train_dev['comment'].values, df_train_dev['y'].values)
print(gs.best_params_)
print(gs.best_score_)

Fitting 1 folds for each of 80 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:  7.0min finished


{'clf__alpha': 1.0, 'vect__min_df': 2, 'vect__ngram_range': (1, 1), 'vect__stop_words': None}
0.6910806174957118


In [68]:
X_dev = df.loc[idx_dev, 'comment'].values
y_dev = df.loc[idx_dev, 'y'].values

y_pred = gs.best_estimator_.predict(X_dev)
confusion_matrix(y_dev, y_pred)

array([[24513,  1737],
       [ 1470,  4408]], dtype=int64)

Interesting that after all that tuning, the best pipeline for the MultinomialNB was basically the default setting with a slight reduction in vocabulary. How much was the vocabulary reduced by?

In [62]:
len(gs.best_estimator_.named_steps['vect'].vocabulary_)

64852

What if we wanted to add a step? Let's add a TfidfTransformer into the pipeline.

In [64]:
from sklearn.feature_extraction.text import TfidfTransformer

pipe = Pipeline([('vect', CountVectorizer(tokenizer=tokenizer)),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB())])
param_grid = {'vect__ngram_range':[(1,1),(1,2)], 'vect__min_df':[1, 2, 5, 10, 20],
              'vect__stop_words':['english', None], 'clf__alpha':[0.1, 0.2, 0.5, 1.0]}
gs = GridSearchCV(pipe, param_grid, scoring='f1', n_jobs=4, cv=ps, verbose=2)
gs.fit(df_train_dev['comment'].values, df_train_dev['y'].values)
print(gs.best_params_)
print(gs.best_score_)

Fitting 1 folds for each of 80 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:  7.1min finished


{'clf__alpha': 0.1, 'vect__min_df': 10, 'vect__ngram_range': (1, 2), 'vect__stop_words': None}
0.6853625170998632


In [65]:
X_dev = df.loc[idx_dev, 'comment'].values
y_dev = df.loc[idx_dev, 'y'].values

y_pred = gs.best_estimator_.predict(X_dev)
confusion_matrix(y_dev, y_pred)

array([[25514,   736],
       [ 1694,  4184]], dtype=int64)